<a href="https://colab.research.google.com/github/kridtapon/Dynamic-Oscillator-Bands-DOB-/blob/main/Dynamic_Oscillator_Bands_(DOB).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.5/527.5 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.43.0
    Uninstalling llvmlite-0.43.0:
      Successfully uninstalled llvmlite-0.43.0
  Attempting uninstall: numba
    Found existing installation: numba 0.60.0
    Uninstalling numba-0.60.0:
      Successfully unins

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
import matplotlib.pyplot as plt

# Function: Chande Momentum Oscillator (CMO)
def chande_momentum_oscillator(df, period=14):
    df['Up'] = np.where(df['Close'] > df['Close'].shift(1), df['Close'] - df['Close'].shift(1), 0)
    df['Down'] = np.where(df['Close'] < df['Close'].shift(1), df['Close'].shift(1) - df['Close'], 0)
    df['SumUp'] = df['Up'].rolling(window=period).sum()
    df['SumDown'] = df['Down'].rolling(window=period).sum()
    df['CMO'] = 100 * (df['SumUp'] - df['SumDown']) / (df['SumUp'] + df['SumDown'])
    return df

# Function: Calculate Bollinger Bands
def calculate_bollinger_bands(df, period=20, std_dev=2):
    df['Middle_Band'] = df['Close'].rolling(window=period).mean()
    rolling_std = df['Close'].rolling(window=period).std()
    df['Upper_Band'] = df['Middle_Band'] + (rolling_std * std_dev)
    df['Lower_Band'] = df['Middle_Band'] - (rolling_std * std_dev)
    return df

# Define the stock symbol and time period
symbol = 'TPL'
start_date = '2020-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)  # Handle missing data

# Calculate indicators: CMO and Bollinger Bands
df = chande_momentum_oscillator(df, period=6)
df = calculate_bollinger_bands(df, period=47, std_dev=1.5)

# Define Entry and Exit Signals
df['Entry'] = (
    (df['CMO'] < -50) &  # CMO is below -50
    (df['Close'] > df['Lower_Band'])  # Price closes above lower Bollinger Band
)

df['Exit'] = (
    (df['CMO'] > 50) &  # CMO is above +50
    (df['Close'] < df['Upper_Band'])  # Price closes below upper Bollinger Band
)

# Convert signals to boolean arrays
entries = df['Entry'].to_numpy()
exits = df['Exit'].to_numpy()

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=entries,
    exits=exits,
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           955154.338685
Total Return [%]                       855.154339
Benchmark Return [%]                   328.047275
Max Gross Exposure [%]                      100.0
Total Fees Paid                      29947.657273
Max Drawdown [%]                        24.388713
Max Drawdown Duration                       234.0
Total Trades                                   33
Total Closed Trades                            32
Total Open Trades                               1
Open Trade PnL                     -201935.591348
Win Rate [%]                               78.125
Best Trade [%]                         134.712905
Worst Trade [%]                        -15.643815
Avg Winning Trade [%]                   14.956912
Avg Losing Trade [%]                    -8.164741


In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
import itertools

# Function: Chande Momentum Oscillator (CMO)
def chande_momentum_oscillator(df, period):
    df['Up'] = np.where(df['Close'] > df['Close'].shift(1), df['Close'] - df['Close'].shift(1), 0)
    df['Down'] = np.where(df['Close'] < df['Close'].shift(1), df['Close'].shift(1) - df['Close'], 0)
    df['SumUp'] = df['Up'].rolling(window=period).sum()
    df['SumDown'] = df['Down'].rolling(window=period).sum()
    df['CMO'] = 100 * (df['SumUp'] - df['SumDown']) / (df['SumUp'] + df['SumDown'])
    return df

# Function: Calculate Bollinger Bands
def calculate_bollinger_bands(df, period, std_dev):
    df['Middle_Band'] = df['Close'].rolling(window=period).mean()
    rolling_std = df['Close'].rolling(window=period).std()
    df['Upper_Band'] = df['Middle_Band'] + (rolling_std * std_dev)
    df['Lower_Band'] = df['Middle_Band'] - (rolling_std * std_dev)
    return df

# Download the data
symbol = 'TPL'
start_date = '2020-01-01'
end_date = '2025-01-01'
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)  # Handle missing data

# Parameter ranges for optimization
cmo_period_range = range(1, 51)  # CMO period
bollinger_period_range = range(1, 51)  # Bollinger Bands period
std_dev_range = np.arange(1.5, 3.0, 0.5)  # Standard deviation: 1.5, 2.0, 2.5

# Optimization
best_params = None
best_performance = -np.inf

for cmo_period, bb_period, std_dev in itertools.product(cmo_period_range, bollinger_period_range, std_dev_range):
    temp_df = df.copy()

    # Calculate indicators
    temp_df = chande_momentum_oscillator(temp_df, period=cmo_period)
    temp_df = calculate_bollinger_bands(temp_df, period=bb_period, std_dev=std_dev)

    # Define Entry and Exit Signals
    temp_df['Entry'] = (
        (temp_df['CMO'] < -50) &
        (temp_df['Close'] > temp_df['Lower_Band'])
    )
    temp_df['Exit'] = (
        (temp_df['CMO'] > 50) &
        (temp_df['Close'] < temp_df['Upper_Band'])
    )

    # Backtest using vectorbt
    portfolio = vbt.Portfolio.from_signals(
        close=temp_df['Close'],
        entries=temp_df['Entry'].to_numpy(),
        exits=temp_df['Exit'].to_numpy(),
        init_cash=100_000,
        fees=0.001
    )

    # Evaluate performance (Total Return as an example)
    performance = portfolio.total_return()

    if performance > best_performance:
        best_performance = performance
        best_params = (cmo_period, bb_period, std_dev)

print(f"Best Parameters: CMO Period = {best_params[0]}, Bollinger Period = {best_params[1]}, Std Dev = {best_params[2]}")
print(f"Best Performance (Total Return): {best_performance}")


[*********************100%***********************]  1 of 1 completed


Best Parameters: CMO Period = 6, Bollinger Period = 47, Std Dev = 1.5
Best Performance (Total Return): 8.551543386846319


In [1]:
import yfinance as yf
import plotly.graph_objects as go
# Download the data
symbol = 'TPL'
start_date = '2020-01-01'
end_date = '2025-01-01'
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)  # Handle missing data

# Create the candlestick chart
fig = go.Figure(data=[go.Candlestick(
    x=df.index,
    open=df['Open'],
    high=df['High'],
    low=df['Low'],
    close=df['Close'],
    increasing_line_color='green',
    decreasing_line_color='red'
)])

# Update layout
fig.update_layout(
    title=f'Candlestick Chart for {symbol}',
    xaxis_title='Date',
    yaxis_title='Price',
    xaxis_rangeslider_visible=False  # Turn off the range slider if not needed
)

# Show the chart
fig.show()

[*********************100%***********************]  1 of 1 completed


In [5]:
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px

# Function: Chande Momentum Oscillator (CMO)
def chande_momentum_oscillator(df, period=14):
    df['Up'] = np.where(df['Close'] > df['Close'].shift(1), df['Close'] - df['Close'].shift(1), 0)
    df['Down'] = np.where(df['Close'] < df['Close'].shift(1), df['Close'].shift(1) - df['Close'], 0)
    df['SumUp'] = df['Up'].rolling(window=period).sum()
    df['SumDown'] = df['Down'].rolling(window=period).sum()
    df['CMO'] = 100 * (df['SumUp'] - df['SumDown']) / (df['SumUp'] + df['SumDown'])
    return df

# Function: Calculate Bollinger Bands
def calculate_bollinger_bands(df, period=20, std_dev=2):
    df['Middle_Band'] = df['Close'].rolling(window=period).mean()
    rolling_std = df['Close'].rolling(window=period).std()
    df['Upper_Band'] = df['Middle_Band'] + (rolling_std * std_dev)
    df['Lower_Band'] = df['Middle_Band'] - (rolling_std * std_dev)
    return df

# Define the stock symbol and time period
symbol = 'TPL'
start_date = '2020-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)  # Handle missing data

# Calculate indicators: CMO and Bollinger Bands
df = chande_momentum_oscillator(df, period=6)
df = calculate_bollinger_bands(df, period=47, std_dev=1.5)

# Plotting Candlestick Chart with Bollinger Bands and CMO

# Create Candlestick chart
candlestick = go.Candlestick(x=df.index,
                             open=df['Open'], high=df['High'],
                             low=df['Low'], close=df['Close'],
                             name="Candlestick")

# Add Bollinger Bands to the plot
upper_band = go.Scatter(x=df.index, y=df['Upper_Band'],
                        line={'color': 'red'}, name="Upper Band")
middle_band = go.Scatter(x=df.index, y=df['Middle_Band'],
                         line={'color': 'green'}, name="Middle Band")
lower_band = go.Scatter(x=df.index, y=df['Lower_Band'],
                        line={'color': 'red'}, name="Lower Band")

# Create CMO plot
cmo_plot = go.Scatter(x=df.index, y=df['CMO'],
                      mode='lines', line={'color': 'blue'}, name="CMO")

# Plot the charts
fig = go.Figure(data=[candlestick, upper_band, middle_band, lower_band, cmo_plot])

# Update layout
fig.update_layout(
    title=f'{symbol} Stock Price with Bollinger Bands and Chande Momentum Oscillator (CMO)',
    xaxis_title='Date',
    yaxis_title='Price',
    template='plotly_dark',
    height=800,
    xaxis_rangeslider_visible=False,  # Hide range slider
    yaxis2=dict(
        title='CMO',
        overlaying='y',
        side='right',
        tickformat=".0f",
        showgrid=False
    ),
)

fig.show()


[*********************100%***********************]  1 of 1 completed
